In [1]:
import os
import pandas as pd
import numpy as np
import re
from collections import defaultdict

# --- Configuração ---
# Pasta onde estão os arquivos originais
input_folder = 'data_rafael_tratado'
# Pasta onde os novos arquivos CSV serão salvos
output_folder = './rafaelDBConcatFolder'

# Nomes das 6 colunas a serem extraídas
column_names = ['Quadrilx', 'Quadrily', 'Quadrilz', 'Joelhox', 'Joelhoy', 'Joelhoz']

# --- Lógica Principal ---

# 1. Cria a pasta de saída se ela não existir
os.makedirs(output_folder, exist_ok=True)
print(f"Arquivos serão salvos em: '{output_folder}'")

# 2. Agrupa os arquivos por voluntário
# Usamos um dicionário onde a chave é o nº do voluntário e o valor é uma lista de seus arquivos
volunteers_files = defaultdict(list)
print(f"🔎 Analisando arquivos em '{input_folder}'...")

for filename in sorted(os.listdir(input_folder)):
    # Usa expressão regular para extrair o número do voluntário de forma segura
    match = re.search(r'Volunt[áa]rio (\d+)_Ex', filename, re.IGNORECASE)
    if match:
        volunteer_num = match.group(1)
        file_path = os.path.join(input_folder, filename)
        volunteers_files[volunteer_num].append(file_path)

if not volunteers_files:
    print("Nenhum arquivo no formato 'Voluntário N_ExM' foi encontrado.")
else:
    print(f"Encontrados dados de {len(volunteers_files)} voluntários: {list(volunteers_files.keys())}")

    # 3. Processa os arquivos de cada voluntário
    for volunteer_num, file_list in volunteers_files.items():
        print(f"\n--- Processando Voluntário nº {volunteer_num} ---")
        
        list_of_dfs_for_volunteer = []
        for file_path in file_list:
            try:
                df = pd.read_csv(file_path, header=None)
                
                # Verifica se há colunas suficientes para a extração
                if df.shape[1] >= 7:
                    # Extrai as 7 últimas colunas, com exceção da última (índices -7 até -1)
                    temp_df = df.iloc[:, -7:-1]
                    temp_df.columns = column_names
                    list_of_dfs_for_volunteer.append(temp_df)
                else:
                    print(f"  Aviso: Arquivo '{os.path.basename(file_path)}' ignorado (menos de 7 colunas).")

            except Exception as e:
                print(f"  Erro ao ler o arquivo '{os.path.basename(file_path)}': {e}")
        
        # 4. Concatena e salva o arquivo final para o voluntário
        if list_of_dfs_for_volunteer:
            # Junta todos os dataframes de exercícios do voluntário
            final_volunteer_df = pd.concat(list_of_dfs_for_volunteer, ignore_index=True)
            
            # Adiciona a coluna de tempo contínua
            timestamps = final_volunteer_df.index * 0.0334
            final_volunteer_df.insert(0, 'timestamp', timestamps)
            
            # Define o nome do arquivo de saída e salva
            output_filename = f'rafaelDBConcatVol{volunteer_num}.csv'
            output_path = os.path.join(output_folder, output_filename)
            
            final_volunteer_df.to_csv(output_path, index=False)
            print(f"✅ Arquivo '{output_filename}' salvo com sucesso ({len(final_volunteer_df)} linhas).")
        else:
            print(f"❌ Nenhum dado válido encontrado para o Voluntário nº {volunteer_num}.")

Arquivos serão salvos em: './rafaelDBConcatFolder'
🔎 Analisando arquivos em 'data_rafael_tratado'...
Encontrados dados de 7 voluntários: ['1', '2', '3', '4', '5', '6', '7']

--- Processando Voluntário nº 1 ---
✅ Arquivo 'rafaelDBConcatVol1.csv' salvo com sucesso (2271 linhas).

--- Processando Voluntário nº 2 ---
✅ Arquivo 'rafaelDBConcatVol2.csv' salvo com sucesso (3426 linhas).

--- Processando Voluntário nº 3 ---
✅ Arquivo 'rafaelDBConcatVol3.csv' salvo com sucesso (3409 linhas).

--- Processando Voluntário nº 4 ---
✅ Arquivo 'rafaelDBConcatVol4.csv' salvo com sucesso (2271 linhas).

--- Processando Voluntário nº 5 ---
✅ Arquivo 'rafaelDBConcatVol5.csv' salvo com sucesso (3426 linhas).

--- Processando Voluntário nº 6 ---
✅ Arquivo 'rafaelDBConcatVol6.csv' salvo com sucesso (4895 linhas).

--- Processando Voluntário nº 7 ---
✅ Arquivo 'rafaelDBConcatVol7.csv' salvo com sucesso (2125 linhas).
